**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [26]:
import keras as keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Dropout

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [4]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [5]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act allows the user to take an action to move from its current state $s$ to another one in the environment, by returning the action to take, using either the learned policy or taking a random action. The ```epsilon``` parameter is essential because it defines the way of tackling the exploration-exploitation dilemma. $$$$
The actions are selected accroding to the learned policy with probability $1-\epsilon$. It corresponds to the **exploitation** part, where we try to maximize the sum of rewards, e.g. taking the best action at this current state. $$$$
The actions are selected randomly with probability $\epsilon$. It corresponds to the **exploration** part, where we try to learn more about the environment, in order to update the policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [6]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [55]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train = 100 # set small when debugging
epochs_test = 11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The ```position``` array corresponds to the grid the rat is on at the current state. It indicates the position of the rat during the game. $$$$
The ```board``` array is used to check on the current state of the game. It indicates whether or not cells have been explored, e.g. if the cheese or the poison have already been eaten. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [15]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """ Returns a random action to take """
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [16]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
        
        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [17]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 16.5/13.0. Average score (3.5)
Win/lose count 7.5/14.0. Average score (-1.5)
Win/lose count 7.0/8.0. Average score (-1.3333333333333333)
Win/lose count 8.0/14.0. Average score (-2.5)
Win/lose count 9.0/11.0. Average score (-2.4)
Win/lose count 12.5/18.0. Average score (-2.9166666666666665)
Win/lose count 7.5/11.0. Average score (-3.0)
Win/lose count 10.5/15.0. Average score (-3.1875)
Win/lose count 11.5/19.0. Average score (-3.6666666666666665)
Win/lose count 7.5/16.0. Average score (-4.15)
Win/lose count 11.0/10.0. Average score (-3.6818181818181817)
Win/lose count 12.0/18.0. Average score (-3.875)
Win/lose count 15.5/15.0. Average score (-3.5384615384615383)
Win/lose count 6.0/15.0. Average score (-3.9285714285714284)
Win/lose count 9.5/9.0. Average score (-3.6333333333333333)
Win/lose count 10.5/26.0. Average score (-4.375)
Win/lose count 7.0/10.0. Average score (-4.294117647058823)
Win/lose count 11.5/21.0. Average score (-4.583333333333333)
Win/lose count 6.5/10.0. 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We have $\forall s \in S, \forall a \in A$:

\begin{equation*}\begin{split}Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&=r(s,a)+E_{p^{\pi}}[\sum_{t=1}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&=r(s,a)+\sum_{s',a'} p(s',a'|s,a) E_{p^{\pi}}[\sum_{t=1}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']\\
&=r(s,a)+ \sum_{s',a'} p(s',a'|s,a) \gamma E_{p^{\pi}} [\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\\
&=\sum_{s',a'} p(s',a'|s,a) (r(s,a) + \gamma  Q^{\pi}(s',a')) \\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{split}\end{equation*}

$$\\ $$
We will assume the second result :
$$ Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] $$

Finally, if we take the following function as the objective function :
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}
\end{equation*}

$\mathcal{L}$ being positive for all $\theta$, if a value $\theta^*$ is such that $\mathcal{L}(\theta^*)=0$, it is then the optimal value.
And in that case we have : 
\begin{equation*}\begin{split}
Q(s,a,\theta^*) &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a',\theta^*)])\\
&=Q^{*}(s,a)
\end{split}\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [20]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        self.indice = 0

    def remember(self, m):
        if (len(self.memory) < self.max_memory):
            self.memory.append(m)
        elif (self.indice < self.max_memory) : 
            self.memory[self.indice] = m
            self.indice += 1
        else:
            self.memory[0] = m
            self.indice = 1
                
    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [21]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [42]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        Q = self.model.predict(np.array([s]))[0]
        return np.argmax(Q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            
            input_states[i] = s_
            target_q[i] = self.model.predict(np.array([s_]))
            
            if game_over_:
                target_q[i][a_] = r_
            else:
                target_q[i][a_] = r_ + self.discount*np.max(self.model.predict(np.array([n_s_]))[0])

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(50))
        model.add(BatchNormalization())
        model.add(Activation("relu"))
        model.add(Dense(50))
        model.add(BatchNormalization())
        model.add(Activation("relu"))
        model.add(Dense(4))
        model.add(Activation("linear"))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [43]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/100 | Loss 0.0284 | Win/lose count 5.0/8.0 (-3.0)
Epoch 001/100 | Loss 0.5121 | Win/lose count 3.5/5.0 (-1.5)
Epoch 002/100 | Loss 0.0578 | Win/lose count 1.5/7.0 (-5.5)
Epoch 003/100 | Loss 0.0614 | Win/lose count 4.0/2.0 (2.0)
Epoch 004/100 | Loss 0.1214 | Win/lose count 2.0/0 (2.0)
Epoch 005/100 | Loss 0.1340 | Win/lose count 6.0/7.0 (-1.0)
Epoch 006/100 | Loss 0.1156 | Win/lose count 11.0/7.0 (4.0)
Epoch 007/100 | Loss 0.0664 | Win/lose count 3.0/3.0 (0.0)
Epoch 008/100 | Loss 0.0628 | Win/lose count 8.0/6.0 (2.0)
Epoch 009/100 | Loss 0.0441 | Win/lose count 5.5/2.0 (3.5)
Epoch 010/100 | Loss 0.0618 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/100 | Loss 0.0340 | Win/lose count 3.0/2.0 (1.0)
Epoch 012/100 | Loss 0.0429 | Win/lose count 5.0/3.0 (2.0)
Epoch 013/100 | Loss 0.0667 | Win/lose count 10.5/2.0 (8.5)
Epoch 014/100 | Loss 0.0496 | Win/lose count 9.0/0 (9.0)
Epoch 015/100 | Loss 0.0344 | Win/lose count 8.0/2.0 (6.0)
Epoch 016/100 | Loss 0.0256 | Win/lose count 3.0/3.0 (

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [50]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), padding="same", activation="relu",input_shape=(5,5,self.n_state)))
        model.add(BatchNormalization())
        model.add(AveragePooling2D())
        model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
        model.add(BatchNormalization())
        model.add(AveragePooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(4))
        model.add(Activation("linear"))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [53]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0164 | Win/lose count 5.5/4.0 (1.5)
Epoch 001/100 | Loss 0.2693 | Win/lose count 4.0/6.0 (-2.0)
Epoch 002/100 | Loss 0.3076 | Win/lose count 2.0/4.0 (-2.0)
Epoch 003/100 | Loss 0.1883 | Win/lose count 7.0/4.0 (3.0)
Epoch 004/100 | Loss 0.0420 | Win/lose count 3.5/0 (3.5)
Epoch 005/100 | Loss 0.0081 | Win/lose count 5.5/2.0 (3.5)
Epoch 006/100 | Loss 0.0037 | Win/lose count 5.5/8.0 (-2.5)
Epoch 007/100 | Loss 0.0032 | Win/lose count 4.5/4.0 (0.5)
Epoch 008/100 | Loss 0.0066 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/100 | Loss 0.0175 | Win/lose count 3.0/3.0 (0.0)
Epoch 010/100 | Loss 0.0164 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/100 | Loss 0.0035 | Win/lose count 5.0/4.0 (1.0)
Epoch 012/100 | Loss 0.0032 | Win/lose count 5.5/0 (5.5)
Epoch 013/100 | Loss 0.0037 | Win/lose count 6.0/3.0 (3.0)
Epoch 014/100 | Loss 0.0162 | Win/lose count 6.0/1.0 (5.0)
Epoch 015/100 | Loss 0.0237 | Win/lose count 4.5/2.0 (2.5)
Epoch 016/100 | Loss 0.0287 | Win/lose count 2.0/1.0 (1.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [63]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/1.0. Average score (4.0)
Win/lose count 4.0/4.0. Average score (2.0)
Win/lose count 10.0/2.0. Average score (4.0)
Win/lose count 3.0/2.0. Average score (3.25)
Win/lose count 7.0/2.0. Average score (3.6)
Win/lose count 8.0/3.0. Average score (3.8333333333333335)
Win/lose count 2.0/3.0. Average score (3.142857142857143)
Win/lose count 5.5/3.0. Average score (3.0625)
Win/lose count 8.5/1.0. Average score (3.5555555555555554)
Win/lose count 8.0/4.0. Average score (3.6)
Win/lose count 6.0/1.0. Average score (3.727272727272727)
Final score: 3.727272727272727
Test of the FC
Win/lose count 4.5/9.0. Average score (-4.5)
Win/lose count 5.5/6.0. Average score (-2.5)
Win/lose count 7.0/7.0. Average score (-1.6666666666666667)
Win/lose count 3.5/3.0. Average score (-1.125)
Win/lose count 3.0/2.0. Average score (-0.7)
Win/lose count 4.0/3.0. Average score (-0.4166666666666667)
Win/lose count 3.5/4.0. Average score (-0.42857142857142855)
Win/lose count 3.0/2.0. Aver

In [64]:
HTML(display_videos('cnn_test10.mp4'))

In [66]:
HTML(display_videos('fc_test10.mp4'))

The oscillations tend to prevent the 'rat' from exploring enough the environment: it becomes too reluctant to explore the grid when the area around him has been cleared. It may be relevant to keep the $\epsilon$-greey exploration to move out from the oscillations. 

When it comes to performances, the CNN gives significantly better results than the FC. Increasing the temperature underlines this gap even more, with the CNN becoming very efficient. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [70]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=10, max_time=500, temperature=0.1)
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
    
    def act(self,action,train=False):
        """This function is very much like the one of the previous Environment class
        but it takes into account new states and the malus grid"""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # Malus consideration
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    
    def reset(self):
        """This function is very much like the one in the previous environment class 
        but it takes into account the new states and the malus grid"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position[self.x,self.y] = 0.1

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [72]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0066 | Win/lose count 4.0/47.400000000000375 (-43.400000000000375)
Epoch 001/100 | Loss 0.4247 | Win/lose count 2.6/48.80000000000042 (-46.20000000000042)
Epoch 002/100 | Loss 0.0129 | Win/lose count 1.2000000000000002/51.50000000000046 (-50.30000000000046)
Epoch 003/100 | Loss 0.0488 | Win/lose count 2.0/55.80000000000047 (-53.80000000000047)
Epoch 004/100 | Loss 0.0041 | Win/lose count 1.6/55.6000000000005 (-54.0000000000005)
Epoch 005/100 | Loss 0.0326 | Win/lose count 1.6/53.60000000000045 (-52.00000000000045)
Epoch 006/100 | Loss 0.0651 | Win/lose count 2.8/54.400000000000446 (-51.60000000000045)
Epoch 007/100 | Loss 0.0134 | Win/lose count 1.6/49.700000000000436 (-48.100000000000435)
Epoch 008/100 | Loss 0.1843 | Win/lose count 4.3999999999999995/53.000000000000455 (-48.600000000000456)
Epoch 009/100 | Loss 0.0085 | Win/lose count 0.4/50.00000000000044 (-49.60000000000044)
Epoch 010/100 | Loss 0.0081 | Win/lose count 2.4/51.50000000000043 (-49.1000000000004

Epoch 087/100 | Loss 0.0133 | Win/lose count 3.9999999999999996/51.10000000000044 (-47.10000000000044)
Epoch 088/100 | Loss 0.0024 | Win/lose count 1.2000000000000002/51.800000000000445 (-50.60000000000044)
Epoch 089/100 | Loss 0.0025 | Win/lose count 0.8/50.90000000000045 (-50.100000000000456)
Epoch 090/100 | Loss 0.0065 | Win/lose count 5.2/52.80000000000046 (-47.600000000000456)
Epoch 091/100 | Loss 0.0041 | Win/lose count 3.1999999999999997/51.300000000000445 (-48.10000000000044)
Epoch 092/100 | Loss 0.0036 | Win/lose count 1.2000000000000002/49.80000000000044 (-48.600000000000435)
Epoch 093/100 | Loss 0.0036 | Win/lose count 6.400000000000001/48.50000000000042 (-42.10000000000042)
Epoch 094/100 | Loss 0.0036 | Win/lose count 0.8/49.90000000000044 (-49.10000000000044)
Epoch 095/100 | Loss 0.0034 | Win/lose count 3.1999999999999997/50.30000000000043 (-47.10000000000043)
Epoch 096/100 | Loss 0.0029 | Win/lose count 3.5999999999999996/50.20000000000043 (-46.60000000000043)
Epoch 097/1

In [73]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 6.0/0. Average score (6.0)
Win/lose count 2.0/3.0. Average score (2.5)
Win/lose count 2.0/3.0. Average score (1.3333333333333333)
Win/lose count 3.5/4.0. Average score (0.875)
Win/lose count 3.0/1.0. Average score (1.1)
Win/lose count 1.5/0. Average score (1.1666666666666667)
Win/lose count 4.0/0. Average score (1.5714285714285714)
Win/lose count 3.5/4.0. Average score (1.3125)
Win/lose count 4.5/0. Average score (1.6666666666666667)
Win/lose count 2.5/0. Average score (1.75)
Win/lose count 5.0/0. Average score (2.0454545454545454)
Final score: 2.0454545454545454


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***